In [ ]:
# Importing required libraries
from bs4 import BeautifulSoup
import requests
import csv
import nltk
import pandas as pd
import re
import numpy as np
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet')
nltk.download('words')
nltk.download('omw-1.4')
import string
from nltk.stem import WordNetLemmatizer
import pickle as pk
from nltk import pos_tag
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
nltk.download('averaged_perceptron_tagger')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Reading the data
df=pd.read_csv("BBC News Train.csv",encoding = "ISO-8859-1")
df=df.drop(['ArticleId'], axis=1)
df.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [ ]:
def removePunctuation(text):
    for i in string.punctuation:
      text=text.replace(i,"")
    return text

df['Text'] = df['Text'].apply(removePunctuation)
df.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in 168m payout eighteen former en...,business


In [ ]:
def removeStopwords(text):
    l=text.split()
    s=""
    for i in l:
      if i not in stop_words :
        s=s+i+" "
    return s[:-1]

stop_words = set(stopwords.words("english"))
df['Text'] = df['Text'].apply(removeStopwords)
df.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens maj...,business
3,lifestyle governs mobile choice faster better ...,tech
4,enron bosses 168m payout eighteen former enron...,business


In [ ]:
def tolower(text):
    text=text.lower()
    return text

df['Text'] = df['Text'].apply(tolower)
df.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens maj...,business
3,lifestyle governs mobile choice faster better ...,tech
4,enron bosses 168m payout eighteen former enron...,business


In [ ]:
def tokenization(text):
  data = word_tokenize(text)
  return data

df['Text'] = df['Text'].apply(tokenization)
df.head()

,Text,Category
0,"[worldcom, exboss, launches, defence, lawyers,...",business
1,"[german, business, confidence, slides, german,...",business
2,"[bbc, poll, indicates, economic, gloom, citize...",business
3,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,"[enron, bosses, 168m, payout, eighteen, former...",business


In [ ]:
def lemmatize(text):

  output = []
  lst = ['a','r','n','v']
  wordNetLemmatizer = WordNetLemmatizer()
  posTag = pos_tag(text)
  for word, tag in posTag:
    pos = tag[0].lower()
    if pos not in lst:
      pos = 'n'
    output.append(wordNetLemmatizer.lemmatize(word,pos))
  return output

df['Text'] = df['Text'].apply(lemmatize)
df.head()

,Text,Category
0,"[worldcom, exboss, launch, defence, lawyer, de...",business
1,"[german, business, confidence, slide, german, ...",business
2,"[bbc, poll, indicate, economic, gloom, citizen...",business
3,"[lifestyle, governs, mobile, choice, faster, w...",tech
4,"[enron, boss, 168m, payout, eighteen, former, ...",business


In [ ]:
def tficf(df):
  classes =df['Category'].unique()
  num_docs = len(df)
  tf = {}
  cf = {}
  for c in classes:
    a = df[df['Category'] == c]
    for words in a['Text']:

      for word in words:
        if word not in tf:
          tf[word] = {}
        if c not in tf[word]:
          tf[word][c] = 0
        tf[word][c] += 1
    for word in tf.keys():
      if word not in cf:
        cf[word] = 0
      if c in tf[word]:
        cf[word] += 1
  tf_icf = {}
  for word in tf.keys():
    tf_icf[word] = {}
    for c in classes:
      a = tf[word].get(c, 0)
      b = cf[word]
      icf = np.log(5/b)
      tf_icf[word][c] = a * icf
  return tf_icf

d=tficf(df)

In [ ]:
print(d)
tf_icf_df=pd.DataFrame(d)
tf_icf_df.head()

{'worldcom': {'business': 86.90964727144141, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'exboss': {'business': 3.2188758248682006, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'launch': {'business': 0.0, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'defence': {'business': 0.0, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'lawyer': {'business': 0.0, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'defend': {'business': 0.0, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'former': {'business': 0.0, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'chief': {'business': 0.0, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'bernie': {'business': 12.875503299472802, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainment': 0.0}, 'ebbers': {'business': 72.42470605953451, 'tech': 0.0, 'politics': 0.0, 'sport': 0.0, 'entertainme

,worldcom,exboss,launch,defence,lawyer,defend,former,chief,bernie,ebbers,...,butter,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
business,86.909647,3.218876,0.0,0.0,0.0,0.0,0.0,0.0,12.875503,72.424706,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
tech,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
politics,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sport,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
entertainment,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,3.218876,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438


In [ ]:
cols=tf_icf_df.columns
ro=df.index
temp_list=[]
X=pd.DataFrame(temp_list,columns=cols,index=ro)
X.head()

,worldcom,exboss,launch,defence,lawyer,defend,former,chief,bernie,ebbers,...,butter,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def split(X,y,i):
  #j=1-i
  lx=len(X)
  ly=len(y)
  nd=int(i*lx)
  x_train=X[:nd]
  x_test=X[nd:]
  y_train=y[:nd]
  y_test=y[nd:]

  return x_train,x_test,y_train,y_test

In [ ]:
from sklearn.naive_bayes import MultinomialNB
x_train, x_test, y_train, y_test = split(X,df['Category'],0.7)
mnb = MultinomialNB()
mnb.fit(x_train, y_train)
y_pred = mnb.predict(x_test)

In [ ]:
y_test=list(y_test)
y_train=list(y_train)

In [ ]:
def score_metrics(y_test,y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred,average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')
  precision = precision_score(y_test, y_pred, average='macro')

  print('Accuracy : ', accuracy)
  print("Precision : ",precision)
  print('Recall : ', recall)
  print('F1score : ', f1)

score_metrics(y_test,y_pred)

Accuracy :  0.9961649089165868
Precision :  0.9963754582555163
Recall :  0.9960855966455956
F1score :  0.9962220895609983
